In [155]:
import pandas as pd
import numpy as np

import warnings

warnings.filterwarnings('ignore')


In [156]:
#BASES
colombia = pd.read_excel('Pico_marcacion_Colombia.xlsx') 
colombia['Pais'] = 'Colombia'

peru     = pd.read_excel('Pico_marcacion_Peru.xlsx')
peru['Pais'] = 'Peru'

ecuador  = pd.read_excel('Pico_marcacion_Ecuador.xlsx')
ecuador['Pais'] = 'Ecuador'

In [157]:
def resultados(df):

   df = df[df[df.columns[6]] == 0]
   df[df.columns[6]] = np.where(df[df.columns[6]]==0,1,0)
   df.columns = df.columns.str.strip()

   #Fecha Pico
   df['Fecha pico'] = pd.to_datetime(df['Date'])

   df['Dia'] = df['Fecha pico'].dt.day
   df['Mes'] = df['Fecha pico'].dt.month

   meses_ingles_a_espanol = {
   1: 'Ene', 2: 'Feb', 3: 'Mar', 4: 'Abr',
   5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Ago',
   9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dic'}

   df['Mes'] = df['Mes'].map(meses_ingles_a_espanol)
   df['Dia'] = [str(x) for x in df['Dia']]
   df['Fecha pico'] = df['Mes'] + '-' + df['Dia']

   #Agrupamiento
   df = df.groupby(by=['Pais','Fecha pico'])[['Llamados a la acción', 'Sanciones',
      'Insultos', 'No es Xenofobia', 'Impact', 'Impressions']].sum()
   

   df.rename(columns = {'No es Xenofobia':'Xenofobia',
                        'Impact':'Total impacto de publicaciones',
                        'Impressions':'Total Impresiones (veces que se vieron estos mensajes)'},inplace=True)
   
   df[['% Llamados a la acción','% Sanciones Judiciales','% Insultos']] = 0     

   df.reset_index(inplace=True)
   
   #Calculo de estadisticos

   df['% Llamados a la acción'] = (df['Llamados a la acción'] / df['Xenofobia']) * 100
   df['% Sanciones Judiciales'] = (df['Sanciones'] / df['Xenofobia']) * 100
   df['% Insultos'] = (df['Insultos'] / df['Xenofobia']) * 100


   #Organizado

   df = df[['Pais','Llamados a la acción', 'Sanciones', 'Insultos','Fecha pico',
            'Total impacto de publicaciones','Total Impresiones (veces que se vieron estos mensajes)','Xenofobia', 
            '% Llamados a la acción', '% Sanciones Judiciales', '% Insultos']]

   df[['% Llamados a la acción', '% Sanciones Judiciales', '% Insultos']] = (df[['% Llamados a la acción', '% Sanciones Judiciales', '% Insultos']]).applymap(lambda x: f'{x:.0f}%')


   return df

In [158]:
# Resultados

colombia,peru,ecuador = resultados(colombia),resultados(peru),resultados(ecuador)

picos = pd.concat([colombia,peru,ecuador])

picos.to_csv('pico_xenofobia_limpio.csv',index=False)